In [112]:
import random
from names import get_full_name
from tqdm import tqdm
from scipy import stats

def expected_win_chance(elo_diff):
    return 100 / (1 + 10**(elo_diff/400))

def expected_draw_chance(elo_diff, elo_level):
    return -0.07067227*elo_diff + 0.01541295*elo_level + 9.57851891

class competition:
    
    def __init__(self, nr_players):
        self.player_poule = [player(skill_level = stats.truncnorm((0 - 0.5) / 0.15, (1 - 0.5) / 0.15, loc=0.5, scale=0.15).rvs(1)[0]) for i in tqdm(range(nr_players))]
        
    def play_rand_matches(self, nr_of_games):
        for game in tqdm(range(nr_of_games)):
            players = random.sample(range(0, len(self.player_poule)), 2)
            
            m = match(self.player_poule[players[0]], self.player_poule[players[1]])
            m.simulate_match()
        
    def show_standings(self):
        sorted_poule = sorted(self.player_poule, key=lambda x: x.rating, reverse=True)
        print("leaderboard standings")
        print("------------------------------------")
        for p in sorted_poule[0:10]:
            print("{} | {:4d} | {:.3f}".format(p.name.ljust(21), round(p.rating), p.skill_level))
        for p in sorted_poule[-11:-1]:
            print("{} | {:4d} | {:.3f}".format(p.name.ljust(21), round(p.rating), p.skill_level))
        print("------------------------------------\n")
    

class match:
    
    def __init__(self, player1, player2):
        self.p1 = player1
        self.p2 = player2
        
        self.es_p1 = 1 / (1 + 10**((player2.rating - player1.rating) / 400))
        self.es_p2 = 1 / (1 + 10**((player1.rating - player2.rating) / 400))
    
    def simulate_match(self):
        win_chances = self.calc_odds(self.p1.skill_level, self.p2.skill_level)

        result_p1 = random.choices([1, 0.5, 0], weights = win_chances, k=1)[0]
        result_p2 = 1-result_p1
        
        self.p1.update_rating(self.es_p1, result_p1)
        self.p2.update_rating(self.es_p2, result_p2)

        self.p1.games_played += 1
        self.p2.games_played += 1

        self.p1.update_k_factor()
        self.p2.update_k_factor
        
        return result_p1

    def calc_odds(self, sl_p1, sl_p2):
        mapped_sl_p1 = 900 + sl_p1 * 2000
        mapped_sl_p2 = 900 + sl_p2 * 2000
        elo_diff = mapped_sl_p2 - mapped_sl_p1

        p1_win_odds = expected_win_chance(elo_diff)
        p2_win_odds = expected_win_chance(-elo_diff)
        draw_odds = expected_draw_chance(abs(elo_diff), max([mapped_sl_p1, mapped_sl_p2]))

        # Correct for imperfect draw odds function
        draw_odds = draw_odds if draw_odds/2 <= min([p1_win_odds, p2_win_odds]) else min([p1_win_odds, p2_win_odds])*2

        p1_win_odds -= draw_odds/2
        p2_win_odds -= draw_odds/2

        return [p1_win_odds, draw_odds, p2_win_odds]



class player:
    
    def __init__(self, rating = 1900, skill_level = 0.5, name = "filler name"):
        self.name = name
        self.rating = rating
        self.skill_level = skill_level
        self.k_factor = 40
        self.games_played = 0
        self.has_hit_2400 = False

    
    def update_rating(self, expected_score, result):
        change = self.k_factor * (result - expected_score)
        self.rating += change

    def update_k_factor(self):
        if self.rating >= 2400 or self.has_hit_2400:
            self.has_hit_2400 = True
            self.k_factor = 10
            return
        
        if self.games_played >= 30:
            self.k_factor = 20
        else:
            self.k_factor = 40
            


In [113]:
c = competition(200000)
c.show_standings()
c.play_rand_matches(10000000)
c.show_standings()

100%|██████████| 1000000/1000000 [00:14<00:00, 69148.49it/s]


leaderboard standings
------------------------------------
filler name           | 2169 | 0.763
filler name           | 2163 | 0.838
filler name           | 2160 | 0.883
filler name           | 2159 | 0.807
filler name           | 2158 | 0.779
filler name           | 2154 | 0.810
filler name           | 2151 | 0.808
filler name           | 2151 | 0.947
filler name           | 2150 | 0.715
filler name           | 2147 | 0.865
filler name           | 1652 | 0.113
filler name           | 1651 | 0.087
filler name           | 1649 | 0.237
filler name           | 1648 | 0.225
filler name           | 1648 | 0.272
filler name           | 1645 | 0.198
filler name           | 1644 | 0.170
filler name           | 1642 | 0.295
filler name           | 1640 | 0.104
filler name           | 1626 | 0.173
------------------------------------



In [10]:
# NOTES
# - gaat er vanuit dat iedereen tegen iedereen speelt
# - match simulatie gaat nog niet goed. de beste moeten meer winnen

# Map to 900-1900-2900, but FIDE rules don't allow elo to be below certain point (I think it is 1000 or 1200)

# K is the development coefficient.
# K = 40 for a player new to the rating list until he has completed events with at least 30 games
# K = 20 as long as a player's rating remains under 2400.
# K = 10 once a player's published rating has reached 2400 and remains at that level subsequently, even if the rating drops below 2400.
# K = 40 for all players until their 18th birthday, as long as their rating remains under 2300.
